# 05 - Model EvaluationAnalyze prediction results and identify areas for improvement.

## Load Predictions

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltfrom pathlib import PathBASE_PATH = Path('..')MODEL_PATH = BASE_PATH / 'model_evaluation'FEATURES_PATH = BASE_PATH / 'features_v2'# Load predictionspred = pd.read_csv(MODEL_PATH / 'predictions_h2.csv')print(f"Loaded predictions: {len(pred):,} rows")pred.head()

## Overall Metrics

In [ ]:
def wmape(actual, predicted):    total = np.sum(np.abs(actual))    if total == 0:        return 999    return 100 * np.sum(np.abs(actual - predicted)) / totaloverall_wmape = wmape(pred['actual'].values, pred['predicted'].values)overall_mae = np.abs(pred['actual'] - pred['predicted']).mean()print("=== Overall Metrics ===")print(f"WMAPE: {overall_wmape:.1f}%")print(f"MAE: {overall_mae:,.0f}")print(f"Total actual: {pred['actual'].sum():,.0f}")print(f"Total predicted: {pred['predicted'].sum():,.0f}")

## Metrics by SKU

In [ ]:
# Calculate per-SKU metricssku_metrics = pred.groupby('sku').apply(    lambda g: pd.Series({        'actual_total': g['actual'].sum(),        'predicted_total': g['predicted'].sum(),        'mae': np.abs(g['actual'] - g['predicted']).mean(),        'wmape': wmape(g['actual'].values, g['predicted'].values),        'n_weeks': len(g)    })).reset_index()print(f"\n=== SKU Metrics Summary ===")print(f"Total SKUs: {len(sku_metrics)}")print(f"\nWMAPE distribution:")print(sku_metrics['wmape'].describe())# Classify by accuracysku_metrics['accuracy_tier'] = pd.cut(    sku_metrics['wmape'],     bins=[0, 20, 40, 60, 100, 1000],    labels=['Excellent (<20%)', 'Good (20-40%)', 'Fair (40-60%)', 'Poor (60-100%)', 'Very Poor (>100%)'])print(f"\nAccuracy Distribution:")print(sku_metrics['accuracy_tier'].value_counts())

## Best & Worst Predictions

In [ ]:
# Top 10 best predictions (lowest WMAPE)print("=== Top 10 Best Predictions ===")best = sku_metrics.nsmallest(10, 'wmape')[['sku', 'wmape', 'actual_total', 'n_weeks']]print(best.to_string(index=False))# Top 10 worst predictionsprint("\n=== Top 10 Worst Predictions ===")worst = sku_metrics.nlargest(10, 'wmape')[['sku', 'wmape', 'actual_total', 'n_weeks']]print(worst.to_string(index=False))

## Visualize Sample Predictions

In [ ]:
# Pick 4 SKUs to visualize (mix of good and bad)sample_skus = list(sku_metrics.nsmallest(2, 'wmape')['sku']) + list(sku_metrics.nlargest(2, 'wmape')['sku'])fig, axes = plt.subplots(2, 2, figsize=(14, 10))axes = axes.flatten()for ax, sku in zip(axes, sample_skus):    sku_data = pred[pred['sku'] == sku].sort_values('year_week')    sku_wmape = sku_metrics[sku_metrics['sku'] == sku]['wmape'].values[0]        ax.plot(range(len(sku_data)), sku_data['actual'], 'b-o', label='Actual')    ax.plot(range(len(sku_data)), sku_data['predicted'], 'r--x', label='Predicted')    ax.set_title(f'SKU: {sku} (WMAPE: {sku_wmape:.1f}%)')    ax.set_xlabel('Week')    ax.legend()plt.tight_layout()plt.show()

## Identify Problem Areas

In [ ]:
# SKUs with high error but high volume (important to fix)high_volume = sku_metrics[sku_metrics['actual_total'] > sku_metrics['actual_total'].median()]problem_skus = high_volume[high_volume['wmape'] > 50].sort_values('actual_total', ascending=False)print(f"=== High-Volume SKUs with Poor Predictions ===")print(f"Found {len(problem_skus)} SKUs with >50% WMAPE and above-median volume")print(problem_skus[['sku', 'wmape', 'actual_total', 'n_weeks']].head(15).to_string(index=False))

## Save Evaluation Results

In [ ]:
# Save SKU metricssku_metrics.to_csv(MODEL_PATH / 'sku_evaluation_metrics.csv', index=False)print(f"✓ Saved sku_evaluation_metrics.csv")# Summary statssummary = {    'total_skus': len(sku_metrics),    'overall_wmape': overall_wmape,    'skus_under_20_wmape': (sku_metrics['wmape'] < 20).sum(),    'skus_under_40_wmape': (sku_metrics['wmape'] < 40).sum(),    'skus_over_100_wmape': (sku_metrics['wmape'] > 100).sum()}print(f"\n=== Summary ===")for k, v in summary.items():    print(f"{k}: {v}")